# Scenario 2 - Passive Manager: Choosing Starting 11 and Captain
## Import required libraries

In [ ]:
import pandas
import numpy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from rsome import ro
from rsome import grb_solver

## 1. Build Model to Solve Starting 11 Problem
### Read dataset

In [ ]:
dataset = pandas.read_csv(filepath_or_buffer='./clean_data/scenario_2/dataset.csv')
dataset

### Declare function

In [ ]:
def onehot_encode(data: pandas.Series) -> (numpy.ndarray, LabelEncoder):
    """
    One-hot encodes data, returns 2D array and encoder that 
    can be used to reverse encoding to retrieve original data.
    Reference: https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
    :param data: 1D data with categorical values.
    :returns encoded: 2D encoded values.
    :returns label_encoder: Encoder, can be used to reverse encoding to retrieve original values.
    
    To reverse encoding: label_encoder.inverse_transform([argmax(encoded[0, :])])
    """
    label_encoder = LabelEncoder()
    int_encoded = label_encoder.fit_transform(y=data)
    int_encoded = int_encoded.reshape(len(int_encoded), 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    encoded = onehot_encoder.fit_transform(X=int_encoded)
    return encoded, label_encoder

### Create model to select starting 11

In [ ]:
def solve_model(dataframe: pandas.DataFrame) -> (numpy.ndarray, numpy.ndarray, float):
    """
    Solves model given data.
    :param dataframe: Dataframe with all players.
    :return: Results from solved model.
    """
    x = dataframe['Total Points'].to_numpy()
    (pos_matrix, _) = onehot_encode(data=dataframe['Position'])
    max_players = 11
    max_captain = 1

    # Position requirement: GK, DEF, FWD
    g = pos_matrix[:,2]
    d = pos_matrix[:,0]
    f = pos_matrix[:,1]
    g_constraint = 1
    d_constraint = 3
    f_constraint = 1
    
    model = ro.Model(name='Starting 11')

    # Define binary decision variables - y: players to choose, z: captain to choose
    y = model.dvar(shape=len(dataframe), vtype='B')
    z = model.dvar(shape=len(dataframe), vtype='B')

    model.max(x @ (y + z))

    model.st(y.sum() == max_players)
    model.st(z.sum() == max_captain)
    model.st(y @ g == g_constraint,
             y @ d >= d_constraint,
             y @ f >= f_constraint)

    model.solve(solver=grb_solver, display=False)

    return y.get(), z.get(), model.get()

### Solve model

In [ ]:
solved_y, solved_z, solved_model = solve_model(dataframe=dataset)
dataset[solved_y == 1]

In [ ]:
dataset[solved_z == 1]

In [ ]:
print("Total Points:", dataset[solved_y == 1]['Total Points'].sum() + dataset[solved_z == 1]['Total Points'].sum())
print("\nCaptain:\n", dataset[solved_z == 1][['Name', 'Total Points', 'Position']])
print("\nStarting 11:\n", dataset[solved_y == 1][['Name', 'Total Points', 'Position']])